In [1]:
import pandas as pd
import ast
import numpy as np

# Load the dataset
file_path = '"C:\\Users\\Admin\\Downloads\\TRADES_CopyTr_90D_ROI.csv"'
data = pd.read_csv("C:\\Users\\Admin\\Downloads\\TRADES_CopyTr_90D_ROI.csv")

# Handle missing values by dropping rows with NaN in Trade_History
clean_data = data.dropna(subset=['Trade_History'])

# Function to parse and extract trade details for each account
def extract_trade_data(trade_history_str):
    try:
        trade_history = ast.literal_eval(trade_history_str)
        parsed_trades = []
        for trade in trade_history:
            parsed_trades.append({
                'time': trade['time'],
                'symbol': trade['symbol'],
                'side': trade['side'],
                'price': trade['price'],
                'fee': trade['fee'],
                'feeAsset': trade['feeAsset'],
                'quantity': trade['quantity'],
                'quantityAsset': trade['quantityAsset'],
                'realizedProfit': trade['realizedProfit'],
                'baseAsset': trade['baseAsset'],
                'qty': trade['qty'],
                'positionSide': trade['positionSide'],
                'activeBuy': trade['activeBuy']
            })
        return parsed_trades
    except:
        return []

# Expand the trades for all accounts
parsed_trades = [trade for trades in clean_data['Trade_History'].apply(extract_trade_data) for trade in trades]
parsed_data = pd.DataFrame(parsed_trades)

# Function to calculate ROI
def calculate_roi(trades):
    total_investment = trades[trades['side'] == 'BUY']['quantity'].sum()
    total_profit = trades['realizedProfit'].sum()
    return total_profit / total_investment if total_investment != 0 else 0

# Function to calculate PnL
def calculate_pnl(trades):
    return trades['realizedProfit'].sum()

# Function to calculate Win Rate
def calculate_win_rate(trades):
    wins = trades[trades['realizedProfit'] > 0].shape[0]
    total_trades = trades.shape[0]
    return wins / total_trades if total_trades != 0 else 0

# Function to calculate Sharpe Ratio
def calculate_sharpe_ratio(trades, risk_free_rate=0):
    returns = trades['realizedProfit'] / trades['quantity']  # Return as profit/investment
    excess_returns = returns - risk_free_rate
    return excess_returns.mean() / returns.std() if returns.std() != 0 else 0

# Function to calculate Maximum Drawdown (MDD)
def calculate_max_drawdown(trades):
    cumulative_returns = (1 + trades['realizedProfit'] / trades['quantity']).cumprod()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    return drawdown.min()  # Minimum drawdown (i.e., maximum loss)

# Apply the functions for each account
grouped = parsed_data.groupby('symbol')

results = grouped.apply(lambda trades: pd.Series({
    'ROI': calculate_roi(trades),
    'PnL': calculate_pnl(trades),
    'WinRate': calculate_win_rate(trades),
    'WinPositions': trades[trades['realizedProfit'] > 0].shape[0],
    'TotalPositions': trades.shape[0],
    'SharpeRatio': calculate_sharpe_ratio(trades),
    'MDD': calculate_max_drawdown(trades)
}))

# Rank accounts by their ROI, PnL, Sharpe Ratio, and MDD (note: MDD is ranked in ascending order)
results['Rank'] = results[['ROI', 'PnL', 'SharpeRatio']].rank(method='min', ascending=False).mean(axis=1) + results['MDD'].rank(method='min')

# Get the top 20 accounts
top_20 = results.sort_values(by='Rank').head(20)

# Save the results to a CSV
top_20.to_csv('top_20_accounts.csv', index=False)

# Output the top 20 accounts
print(top_20)

                    ROI            PnL   WinRate  WinPositions  \
symbol                                                           
ENSUSDT        0.023618   17881.242209  0.392371        2047.0   
JASMYUSDT      0.017453   44192.855115  0.414906        3908.0   
ONDOUSDT       0.014685    5154.675947  0.321796         688.0   
1000FLOKIUSDT  0.011087    4047.785940  0.428808        1295.0   
LPTUSDT        0.018345    5224.597617  0.429133        1529.0   
CKBUSDT        0.021235    2279.231149  0.367169         633.0   
JTOUSDT        0.018110    3739.273679  0.444921         622.0   
ARUSDT         0.013051    3824.130176  0.391903        1452.0   
ENAUSDT        0.009610    4825.108939  0.420274         767.0   
DOGEUSDT       0.027382   28221.565316  0.338614        1026.0   
1000PEPEUSDT   0.005187   27934.409449  0.410275        8441.0   
ORDIUSDT       0.008676   24248.854630  0.414863        4466.0   
SSVUSDT        0.019498    5054.080730  0.485507        1005.0   
PENDLEUSDT

C:\Users\Admin\AppData\Local\Temp\ipykernel_21784\4289218637.py:73: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results = grouped.apply(lambda trades: pd.Series({
